## Solving big systems partly (sparse matrix -> scipy.sparse.linalg.eigsh)

In [1]:
import functions as f #self-defined functions in functions.py
import kwant
import tinyarray as tiny
import numpy as np 
from numpy import linalg as LA
import scipy.sparse.linalg as sla
from types import SimpleNamespace
import holoviews as hv
from holoviews import opts
from matplotlib import pyplot as plt
hv.extension('matplotlib', logo=False) #BOKEH DOESNT SUPPORT LATEX IN LABELS
# hv.extension('bokeh', logo=False)
import time

pauli=f.pauli

def disk(position): 
    x,y = position
    return x**2 + y**2 < radius**2

def magn_texture(position,azi_winding, radi_winding):
    x,y = position
    theta = np.arctan2(x,y)
    q = azi_winding
    p = radi_winding
    R = radius
    r = np.sqrt(x**2 + y**2)
    B = [np.sin(np.pi*p*(r/R))*np.cos(q*theta), np.sin(np.pi*p*(r/R))*np.sin(q*theta), np.cos(np.pi*p*(r/R))]
    return B
    
def hopping(position1,position2,t): #define the hopping terms in your system
    return -t*pauli.szs0

C:\ProgramData\Anaconda3\envs\lps\lib\site-packages\kwant\solvers\default.py:18: RuntimeWarning: MUMPS is not available, SciPy built-in solver will be used as a fallback. Performance can be very poor in this case.
  "Performance can be very poor in this case.", RuntimeWarning)


In [10]:
#define a Boolean function to shape your system
def onsite(site, t, mu, j, azi_winding, radi_winding, delta): #define a function to determine the onsite energy term of the Hamiltonian
    position = site.pos #site is a class! Apart from real space position contains the type of atom (to which family it belongs, how many orbitals etc)
#     B = magn_texture(position,azi_winding,radi_winding) #calculate direction of magnetic field at position (x,y)
#     skyrmion_interaction = j*(B[0]*pauli.s0sx + B[1]*pauli.s0sy + B[2]*pauli.s0sz)
    return 4*t*pauli.szs0 - mu*pauli.szs0 + delta*pauli.sxs0 + j*pauli.s0sz

radius = 100
sys = kwant.Builder() #initialize your system
sqlat = kwant.lattice.square()

sys[sqlat.shape(disk,(0,0))]= onsite
sys[sqlat.neighbors()]= hopping

sys= sys.finalized()
# system_plot = kwant.plot(sys)

In [11]:
params = dict(t=1, mu=1, j=1, delta=0, azi_winding=1, radi_winding=1)
ham = sys.hamiltonian_submatrix(params=params, sparse=True)

In [12]:
t1 = time.time()
eVal, eVec = sla.eigsh(ham,k=30)
t2 = time.time()
print(t2-t1)

149.1892204284668


In [14]:
hv.Scatter((range(len(eVal)),eVal))

:Scatter   [x]   (y)

In [ ]:
def get_spectrum(system, params, timing=False, plot=False):
    #system has to be a finalized Kwant system
    ham = system.hamiltonian_submatrix(params=params)
    
    if timing:        
        t1 = time.time()
        eVal =LA.eigvalsh(ham)
        t2 = time.time()
        print('Hamiltonian size = {0:d}x{0:d} \nSolving eigenvalues took {1:.3f}s'.format(len(eVal),t2-t1))
    else:
        eVal =LA.eigvalsh(ham)
    
    if plot:
        fig, ax = plt.subplots(figsize=(10,6))
        fig.suptitle("Spectrum", fontsize=17)
        plt.plot(eVal)
        ax.set_xlabel('x', fontsize=15)
        ax.set_ylabel('$\epsilon$', fontsize=15)
        params_box(ax, params)    
        
    return eVal